In [2]:
import pandas as pd
import numpy as np

In [3]:
oil_df=pd.read_csv("oil_gas_dataset.csv")

In [4]:
oil_df.head()

,Symbol,Date,Open,High,Low,Close,Volume,Currency
0,Brent Oil,2000-01-04,23.90,24.70,23.89,24.39,32509,USD
1,Brent Oil,2000-01-05,24.25,24.37,23.70,23.73,30310,USD
2,Brent Oil,2000-01-06,23.55,24.22,23.35,23.62,44662,USD
3,Brent Oil,2000-01-07,23.57,23.98,23.05,23.09,34826,USD
4,Brent Oil,2000-01-10,23.04,23.78,23.04,23.73,26388,USD


In [5]:
oil_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23024 entries, 0 to 23023
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Symbol    23024 non-null  object 
 1   Date      23024 non-null  object 
 2   Open      23024 non-null  float64
 3   High      23024 non-null  float64
 4   Low       23024 non-null  float64
 5   Close     23024 non-null  float64
 6   Volume    23024 non-null  int64  
 7   Currency  23024 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.4+ MB


In [6]:
gdp_df=pd.read_csv("sectoral_gdp.csv")

In [7]:
gdp_df.head()

,Quarter/Year,Unnamed: 1,2012-1,2012-2,2012-3,2012-4,2013-1,2013-2,2013-3,2013-4,...,2021-1,2021-2,2021-3,2021-4,2022-1,2022-2,2022-3,2022-4,2023-1,2023-2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Industry,ISIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Agriculture, forestry and fishing",A,289.5,282.1,284.0,287.5,279.9,270.3,269.8,271.5,...,256.4,238.6,219.5,218.7,204.5,208.5,196.3,204.7,196.2,205.2
3,Mining and quarrying,B,"9,212.6","8,893.6","8,773.2","8,486.3","9,024.4","9,047.8","9,031.8","8,871.1",...,"5,901.5","5,708.6","5,589.9","5,669.7","5,573.5","5,683.4","6,079.8","5,670.9","5,630.0","5,572.6"
4,Manufacturing,C,"7,888.0","7,847.7","7,768.4","7,030.1","7,692.1","7,710.2","7,516.2","7,537.2",...,"5,980.1","6,066.7","6,100.6","6,169.4","6,308.3","6,324.1","6,572.4","6,419.2","6,189.8","6,276.9"


In [8]:
gdp_df=gdp_df.transpose()

In [9]:
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, Quarter/Year to 2023-2
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       0 non-null      object
 1   1       2 non-null      object
 2   2       48 non-null     object
 3   3       48 non-null     object
 4   4       48 non-null     object
 5   5       1 non-null      object
 6   6       48 non-null     object
 7   7       48 non-null     object
 8   8       48 non-null     object
 9   9       48 non-null     object
 10  10      48 non-null     object
 11  11      48 non-null     object
 12  12      48 non-null     object
 13  13      48 non-null     object
 14  14      48 non-null     object
 15  15      48 non-null     object
 16  16      48 non-null     object
 17  17      48 non-null     object
 18  18      48 non-null     object
 19  19      1 non-null      object
 20  20      48 non-null     object
 21  21      48 non-null     object
 22  22      48 non-nul

In [10]:
gdp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
Quarter/Year,NaN,Industry,"Agriculture, forestry and fishing",Mining and quarrying,Manufacturing,Of which:,"Food, beverages and tobacco products","Textiles, clothing, leather, wood, paper and p...",Petroleum and chemical products,Other manufactured products,...,Of which,Crude Oil Exploration & Extraction,Condensate Extraction,Natural Gas Exploration & Extraction,Asphalt,Petroleum support services,Refining (incl. LNG),Manufacture of Petrochemicals,Petroleum and natural gas distribution,Source: Central Statistical Office
Unnamed: 1,NaN,ISIC,A,B,C,NaN,CA,CB-C,CD-E,CF-M,...,NaN,B1,B2,B3,B4,B5,C1,C2,G1,NaN
2012-1,NaN,NaN,289.5,"9,212.6","7,888.0",NaN,"1,199.1",231.6,"5,959.2",498.1,...,NaN,"3,027.1","1,344.6","4,074.5",55.2,679.2,"2,638.6","3,091.3","1,290.3",NaN
2012-2,NaN,NaN,282.1,"8,893.6","7,847.7",NaN,"1,200.2",280.6,"5,811.6",555.3,...,NaN,"3,182.4","1,100.6","3,924.4",55.2,630.4,"2,607.8","3,027.3","1,297.9",NaN
2012-3,NaN,NaN,284.0,"8,773.2","7,768.4",NaN,"1,227.5",263.9,"5,744.2",532.8,...,NaN,"3,188.1",970.9,"4,083.1",55.2,447.1,"2,611.7","2,932.7","1,283.5",NaN


Preprocessing decisions are based on the following observations:<br>
The first row of this dataset contains industry labels.<br>
The second row is a code for each industry.<br>
There are many NaN columns (0,1,5,19,31,40).<br>
Multi-letter codes (row 2) contain a combination of industry products. Our research focuses on the contribution of individual sectors. Thus these rows are not needed.

In [11]:
gdp_clean=gdp_df.copy()

In [12]:
#extract dates column
dates=pd.DataFrame(gdp_clean[0])
dates.info()
print(dates.head())
print(dates.iloc[5])


<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, Quarter/Year to 2023-2
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       0 non-null      object
dtypes: object(1)
memory usage: 1.8+ KB
                0
Quarter/Year  NaN
Unnamed: 1    NaN
2012-1        NaN
2012-2        NaN
2012-3        NaN
0    NaN
Name: 2012-4, dtype: object


In [13]:
#clean dates data
years=[]
quarters=[]
for i in range(2,len(dates)):
    date=dates.iloc[i].name

    #convert to string and remove trailing whitespace
    date=str(date).strip()

    #split into year and quarter
    year, quarter = date.split("-")
    years.append(int(year))
    quarters.append(int(quarter))

dates_df=pd.DataFrame({'Year':years, 'Quarter':quarters})
dates_df.head()


,Year,Quarter
0,2012,1
1,2012,2
2,2012,3
3,2012,4
4,2013,1


In [16]:
# set column names to industry
gdp_clean.columns=gdp_clean.iloc[0]
gdp_clean=gdp_clean[1:]



In [17]:
#extract and clean total gdp column
total_gdp=gdp_clean['GDP at basic prices']
values=[]

for val in total_gdp[1:]:   #exclude nan in first row
    amount=float(val.replace(",",""))
    values.append(amount)

total_gdp_df=pd.DataFrame({'Total GDP':values})
total_gdp_df.head()

,Total GDP
0,41793.4
1,42040.3
2,42450.8
3,41692.8
4,42009.3


In [82]:
#extract and clean industry columns
industries=[]
values=[]
for col in gdp_clean.columns:
    data=gdp_clean[col].iloc[0]
    data=str(data).strip()
    if len(data)==1:
        industries.append(col)

industries_df=pd.DataFrame(gdp_clean[industries][1:])


industries_df = industries_df.reset_index(drop=True) 
industries_df


Quarter/Year,"Agriculture, forestry and fishing",Mining and quarrying,Manufacturing,Electricity and gas,Water supply and sewerage,Construction,Trade and repairs,Transport and storage,Accommodation and food services,Information and communication,Financial and insurance activities,Real estate activities,"Professional, scientific and technical services",Administrative and support services,Public administration,Education,Human health and social work,"Arts, entertainment and recreation",Other service activities,Domestic services
0,289.5,"9,212.6","7,888.0","1,309.3",512.3,"2,155.9","9,179.4","1,314.3",563.6,945.3,"2,333.3",753.3,604.0,779.6,"2,687.4",903.4,147.4,94.4,80.5,39.8
1,282.1,"8,893.6","7,847.7","1,299.3",509.3,"2,199.7","9,855.2","1,316.1",523.3,956.9,"2,304.2",754.7,544.2,801.2,"2,682.8",905.4,147.1,94.1,83.2,40.0
2,284.0,"8,773.2","7,768.4","1,327.9",513.2,"2,216.4","10,018.6","1,501.2",546.6,974.9,"2,329.7",756.6,602.5,856.3,"2,701.3",914.3,147.9,94.7,82.9,40.1
3,287.5,"8,486.3","7,030.1","1,229.4",522.3,"2,037.7","10,746.8","1,234.4",506.4,981.3,"2,360.5",757.4,619.1,921.5,"2,686.9",917.6,147.7,94.5,85.4,40.3
4,279.9,"9,024.4","7,692.1","1,329.9",485.1,"2,246.7","9,442.0","1,455.2",555.3,978.9,"2,339.3",760.1,446.9,"1,020.8","2,658.4",924.4,148.0,94.7,87.0,40.4
5,270.3,"9,047.8","7,710.2","1,308.6",493.5,"2,383.2","9,883.0","1,331.6",544.9,986.0,"2,145.8",760.9,647.5,"1,076.0","2,673.1",926.1,148.4,95.0,84.5,40.5
6,269.8,"9,031.8","7,516.2","1,296.0",511.6,"2,241.9","10,722.2","1,544.6",558.0,"1,002.8","2,174.6",772.3,683.5,"1,090.1","2,688.7",934.9,148.5,95.0,84.3,40.7
7,271.5,"8,871.1","7,537.2","1,350.7",500.8,"2,237.0","12,896.0","1,612.0",522.6,"1,030.6","2,174.3",769.1,712.7,"1,051.5","2,748.0",938.3,148.6,95.1,87.0,40.8
8,256.5,"8,787.9","7,451.5","1,374.4",473.2,"2,305.2","11,239.8","1,438.1",520.9,"1,003.8","2,203.9",772.2,698.5,973.3,"2,737.0",945.4,148.6,95.1,85.5,40.9
9,256.1,"8,776.8","7,495.6","1,292.5",450.1,"2,239.0","12,385.1","1,439.9",531.5,"1,020.1","2,226.9",775.3,668.4,929.2,"2,770.8",949.9,148.9,95.3,85.1,41.1
